In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input, Concatenate, UpSampling2D

In [2]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.3.0
Default GPU Device: /device:GPU:0


In [3]:
img_height, img_width = 256, 256

def normalize(input_image):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image


def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def decode_mask_img(img):
  # convert the compressed string to a 2D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=1)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])




In [4]:
class TableNet:
  @staticmethod
  def build_table_decoder(inputs, pool3, pool4):
    x = Conv2D(512, (1, 1), activation = 'relu', name='conv7_table')(inputs)
    x = UpSampling2D(size=(2, 2))(x)

    concatenated = Concatenate()([x, pool4])

    # concatenated = concatenate([x, pool4])

    x = UpSampling2D(size=(2,2))(concatenated)
    
    concatenated = Concatenate()([x, pool3])

    x = UpSampling2D(size=(2,2))(concatenated)
    x = UpSampling2D(size=(2,2))(x)

    last = tf.keras.layers.Conv2DTranspose(
      3, 3, strides=2,
      padding='same', name='table_output') 
    
    x = last(x)

    return x

  @staticmethod
  def build_column_decoder(inputs, pool3, pool4):
    
    x = Conv2D(512, (1, 1), activation = 'relu', name='block7_conv1_column')(inputs)
    x = Dropout(0.8, name='block7_dropout_column')(x)

    x = Conv2D(512, (1, 1), activation = 'relu', name='block8_conv1_column')(x)
    x = UpSampling2D(size=(2, 2))(x)

    concatenated = Concatenate()([x, pool4])

    # concatenated = concatenate([x, pool4])

    x = UpSampling2D(size=(2,2))(concatenated)
    
    concatenated = Concatenate()([x, pool3])

    x = UpSampling2D(size=(2,2))(concatenated)
    x = UpSampling2D(size=(2,2))(x)

    last = tf.keras.layers.Conv2DTranspose(
      3, 3, strides=2,
      padding='same', name='column_output') 
    
    x = last(x)

    return x  

  @staticmethod
  def vgg_base(inputs):
    base_model = tf.keras.applications.vgg19.VGG19(
        input_shape=[256, 256, 3],
        include_top=False, weights='imagenet')
    
    layer_names = ['block3_pool', 'block4_pool', 'block5_pool']
    layers = [base_model.get_layer(name).output for name in layer_names]

    pool_layers_model = Model(inputs=base_model.input, outputs=layers, name='VGG-19')
    pool_layers_model.trainable = False

    return pool_layers_model(inputs)
  
  @staticmethod
  def build():
    inputShape = (256, 256, 3)

    inputs = Input(shape=inputShape, name='input')

    pool_layers = TableNet.vgg_base(inputs)

    x = Conv2D(512, (1, 1), activation = 'relu', name='block6_conv1')(pool_layers[2])
    x = Dropout(0.8, name='block6_dropout1')(x)
    x = Conv2D(512, (1, 1), activation = 'relu', name='block6_conv2')(x)
    x = Dropout(0.8, name = 'block6_dropout2')(x)
    
    table_mask = TableNet.build_table_decoder(x, pool_layers[0], pool_layers[1])
    #column_mask = TableNet.build_column_decoder(x, pool_layers[0], pool_layers[1])

    model = Model(
              inputs=inputs,
                    outputs=[table_mask],
                    name="tablenet")
    
    return model

In [5]:
import pydot
import graphviz

In [6]:
model = TableNet.build()


In [7]:
tf.keras.utils.plot_model(model, show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [8]:
losses = {
    "table_output": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #,column_output": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
}

lossWeights = {"table_output": 1.0
               #, "column_output": 1.0
              }

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, epsilon=1e-08),
              loss=losses,
              metrics=['accuracy'],
              loss_weights=lossWeights)


In [9]:
#model.summary()

In [10]:
list_ds = tf.data.Dataset.list_files('data/labels/*.xml')

# list_ds = tf.data.Dataset.list_files('/content/drive/My Drive/datasets/flipkart_grid/Marmot_data/*.xml')

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [12]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="data/images/",target_size=(224,224))


Found 0 images belonging to 0 classes.


In [13]:
from os import listdir
from xml.etree import ElementTree
from mrcnn.utils import Dataset
class TableDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        
        # Add classes. We have only one class to add.
        self.add_class("dataset", 1, "Table")
        
        # define data locations for images and annotations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/labels/'
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4]
            
            # skip bad images
            if image_id in ['00090']:
                continue
            # skip all images after 150 if we are building the train set
            if is_train and int(image_id) >= 200:
                continue
            # skip all images before 150 if we are building the test/val set
            if not is_train and int(image_id) < 200:
                continue
            
            # setting image file
            img_path = images_dir + filename
            
            # setting annotations file
            ann_path = annotations_dir + image_id + '.xml'
            
            # adding images and annotations to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    # load the masks for an image
    """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
     """
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        
        # define anntation  file location
        path = info['labels']
        
        # load XML
        boxes, w, h = self.extract_boxes(path)
       
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('Tables'))
        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    #Return the path of the image."""
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

In [14]:
# prepare train set
train_set = TableDataset()
train_set.load_dataset('data/', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = TableDataset()
test_set.load_dataset('data/', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 191
Test: 148


In [15]:
#from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = ModelCheckpoint("tablenet.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [16]:
#arly = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [17]:
#hist = model.fit_generator(steps_per_epoch=100,generator=train_set, validation_data= test_set, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

In [18]:
from IPython.display import clear_output

class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="mymodel_{epoch}",
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    )

    early_stoping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        verbose=1,
    )



In [19]:
import pandas as pd

In [31]:
table=pd.read_csv('datatesttrain/records.csv')


In [32]:
table.head(1)

,Unnamed: 0,filename,imagefilepath,targetfilepath,imagesize,tablecount,tables,bndboxlim
0,0,00001.PNG,data/images/00001.PNG,data/labels/00001.xml,"['1126', '1596', '3']",1,"[OrderedDict([('name', 'Table'), ('pose', 'Uns...","[OrderedDict([('xmin', '15'), ('ymin', '442'),..."


In [22]:
def process_path(file_path):
  #print(file_path)
  file_path = tf.strings.regex_replace(file_path, '.xml', '.PNG')
  file_path = tf.strings.regex_replace(file_path, 'labels', 'images')
  table_mask_file_path = tf.strings.regex_replace(file_path, 'labels', 'table_mask')
  #column_mask_file_path = tf.strings.regex_replace(mask_file_path, 'Marmot_data', 'column_mask')
  img = normalize(decode_img(tf.io.read_file(file_path)))
  table_mask = normalize(decode_mask_img(tf.io.read_file(table_mask_file_path)))
  #column_mask = normalize(decode_mask_img(tf.io.read_file(column_mask_file_path)))

  return {"input":img,"table_output" : table_mask}

In [23]:
img=process_path('data/labels/00001.xml')

In [33]:

DATASET_SIZE = len(list(list_ds))
train_size = int(0.9 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

In [34]:
def process_path1(file_path):
    
  #print(file_path)
    file_path = tf.strings.regex_replace(file_path, '.xml', '.PNG')
    file_path = tf.strings.regex_replace(file_path, 'labels', 'images')
    table_mask_file_path = tf.strings.regex_replace(file_path, 'labels', 'table_mask')
    #column_mask_file_path = tf.strings.regex_replace(mask_file_path, 'Marmot_data', 'column_mask')
    
    img = normalize(decode_img(tf.io.read_file(file_path)))
    print(img.shape)
    table_mask = normalize(decode_mask_img(tf.io.read_file(table_mask_file_path)))
    #column_mask = normalize(decode_mask_img(tf.io.read_file(column_mask_file_path)))

    return img,{"table_output" : table_mask}

dataset = list_ds.map(process_path1)

train = dataset.take(train_size)
test = dataset.skip(train_size)
TRAIN_LENGTH = len(list(train))
BATCH_SIZE = 2
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE


#train = train.shuffle(BUFFER_SIZE)

#train = train.map(process_path1)
                      #num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 

#test = test.map(process_path)

train_dataset = train.batch(BATCH_SIZE).repeat().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

(256, 256, None)


In [35]:
(train_dataset.take(3))

<TakeDataset shapes: ((None, 256, 256, None), {table_output: (None, 256, 256, 1)}), types: (tf.float32, {table_output: tf.float32})>

In [36]:
def process_path(file_path):
  #print(file_path)
  file_path = tf.strings.regex_replace(file_path, '.xml', '.PNG')
  file_path = tf.strings.regex_replace(file_path, 'labels', 'images')
  table_mask_file_path = tf.strings.regex_replace(file_path, 'labels', 'table_mask')
  #column_mask_file_path = tf.strings.regex_replace(mask_file_path, 'Marmot_data', 'column_mask')
  img = normalize(decode_img(tf.io.read_file(file_path)))
  table_mask = normalize(decode_mask_img(tf.io.read_file(table_mask_file_path)))
  #column_mask = normalize(decode_mask_img(tf.io.read_file(column_mask_file_path)))

  return {"input":img,
          "table_output" : table_mask
         }

In [51]:
def readimage(file_path):
    file_path =file_path.replace('.xml', '.PNG')
    file_path = file_path.replace('labels', 'images')
    img = normalize(decode_img(tf.io.read_file(file_path)))
    return img


def readmask(file_path):
    file_path = file_path.replace('.xml', '.jpeg')
    file_path = file_path.replace('labels', 'tablemask')
    #print(file_path)
    img = normalize(decode_mask_img(tf.io.read_file(file_path)))
    return img

    

In [55]:
decode_img(tf.io.read_file('data/tablemask/00001.jpeg'))

<tf.Tensor: shape=(256, 256, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>

In [49]:
data=pd.DataFrame()
data['input']=table['targetfilepath'].apply(readimage) 

In [57]:
data['table_output'].iloc[1]

<tf.Tensor: shape=(256, 256, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>

In [52]:
data['table_output']=table['targetfilepath'].apply(readmask) 

In [43]:
train_dataset=data[:10]
test_dataset=data[10:15]

In [58]:
with tf.device("/gpu:0"):
    EPOCHS = 5000
    VAL_SUBSPLITS = 5
    VALIDATION_STEPS = test_size//BATCH_SIZE//VAL_SUBSPLITS


    model_history = model.fit(train_dataset,batch_size=8,epochs=EPOCHS,
                              steps_per_epoch=STEPS_PER_EPOCH
                              #validation_steps=VALIDATION_STEPS,
                              #validation_data=test_dataset
                              #callbacks=[DisplayCallback(), model_checkpoint]
                             )

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

In [ ]:
len(train_dataset)

In [ ]:
train_dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="data/",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="data/cats_and_dogs_filtered/validation", target_size=(224,224))

In [ ]:
#model_history = model.fit(x=train_dataset,epochs=EPOCHS,
 #                         steps_per_epoch=STEPS_PER_EPOCH
                          
                          #,validation_steps=VALIDATION_STEPS,
                          #validation_data=test_dataset
                          #,callbacks=[DisplayCallback(), model_checkpoint]
                         #)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("table.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto',save_freq =1)
early = EarlyStopping(monitor='t_acc', min_delta=0, patience=20, verbose=1, mode='auto')
with tf.device("/gpu:0"):
    hist = model.fit(steps_per_epoch=100,generator=train_dataset, validation_data=test_dataset, validation_steps=10,epochs=100,callbacks=[checkpoint,early])